In [1]:
import pandas as pd
import numpy as np
import pymongo
from pprint import pprint
import bookclub_test_data as td

In [2]:
def first_day():
    now = pd.Timestamp.now() - pd.offsets.MonthBegin(1)
    return now.strftime('%Y-%m-%d')

In [3]:
def last_day():
    now = pd.Timestamp(first_day()) + pd.offsets.MonthBegin(1)
    return now.strftime('%Y-%m-%d')

In [4]:
between_dates_query = {"date":{"$gte": first_day(), "$lt": last_day()}}
vote_count_pipline = [{ "$addFields": {"vote_count": { "$size": "$votes" } } }]
reset_vote = [{"$set": {"votes": [], "vote_count": 0}}]
# vote = [{"$push": {"votes": }}]
find = {"_id": 0}
find_titles = {"title": 1}
find_books = { "_id": 0, "date": 0, "votes": 0}
find_nice = { "_id": 0, "votes": 0}
sort_books = [("vote_count", -1), ("date", 1)] # first highest vote count then lowest date

In [5]:
class Collection:
    def __init__(self, collection):
        self.client = pymongo.MongoClient("mongodb://localhost:27017/")
        self.mydb = self.client["bookclub"]
        self.collection = self.mydb[collection]

    def add(self, book):
        self.collection.insert_one(book)
    
    def add_many(self, books):
        self.collection.insert_many(books)
    
    def add_vote(self, title, username):
        self.collection.update_one({"title": title}, [{"$push": {"votes": usermame}}])
        
    def add_votes(self, title, usernames):
        self.collection.update_one({"title": title}, [{"$push": {"votes": {"$each": usermames}}}])

    def delete(self, book):
        self.collection.delete_one(book)

    def count_votes(self):
        self.collection.update_many({}, vote_count_pipline)
        
    def reset_votes(self):
        self.collection.update_many({}, reset_vote)

    def drop(self):
        self.collection.drop()

    def find_all(self, query, fields, sort):
        for book in self.collection.find(query,fields).sort(sort):
            yield book
            
    def find(self, query, fields, sort, total):
        for book in self.collection.find(query,fields).sort(sort).limit(total):
            yield book

In [6]:
incomming = Collection("incomming")

In [7]:
results = incomming.find_all({},{},[("date", -1)])

In [8]:
# pprint([result for result in results])

In [27]:
class Manager:
    def __init__(self):
        self.incomming = Collection("incomming")
        self.potential = Collection("potential")
        self.buffer = Collection("buffer")
        self.total = 3
        self.book_of_month = ""
    
    def incomming_books(self, books):
        self.incomming.add_many(books)
        self.incomming.count_votes()
        return [book for book in self.incomming.find(between_dates_query, find_nice, sort_books, self.total)]
    
    def potential_books(self, books):
        self.potential.add_many(books)
        self.potential.reset_votes()
        random_book = np.random.choice([book for book in self.incomming.find_all({},{},[("date", -1)])])
        self.book_of_month = random_book
        self.potential.delete(random_book)
        return self.book_of_month
       
    def potential_remove(self):
        # this is also for generating tests > wil be moved to a dedicated testfile
        for title in self.potential.find_all({}, find_titles, sort_books):
            self.potential.add_votes(title["title"], td.generate_title_votes(1))
        self.potential.count_votes()
        for book in self.find(between_dates_query, find_nice, sort_books, self.total):
            self.buffer.add(book)
            self.potential.delete(book)
            
    def drop_all(self):
        bookclub.incomming.drop()
        bookclub.potential.drop()
        bookclub.buffer.drop()

In [28]:
bookclub = Manager()

In [29]:
books = td.generate_books(10, 1)
top_incomming = bookclub.incomming_books(books)

In [30]:
bookclub.potential_books(top_incomming)

{'_id': ObjectId('63c33fa19f340ce701cd82c6'),
 'title': 'title-1',
 'author': 'author-1',
 'pages': 234,
 'user': 'user-1',
 'votes': [],
 'date': '2023-01-25',
 'vote_count': 0}

In [31]:
for x in bookclub.incomming.collection.find({}):
    print(x)

{'_id': ObjectId('63c33fa19f340ce701cd82c5'), 'title': 'title-0', 'author': 'author-0', 'pages': 321, 'user': 'user-0', 'votes': ['user-0'], 'date': '2023-01-04', 'vote_count': 1}
{'_id': ObjectId('63c33fa19f340ce701cd82c6'), 'title': 'title-1', 'author': 'author-1', 'pages': 234, 'user': 'user-1', 'votes': [], 'date': '2023-01-25', 'vote_count': 0}
{'_id': ObjectId('63c33fa19f340ce701cd82c7'), 'title': 'title-2', 'author': 'author-2', 'pages': 419, 'user': 'user-2', 'votes': ['user-7', 'user-3'], 'date': '2023-01-10', 'vote_count': 2}
{'_id': ObjectId('63c33fa19f340ce701cd82c8'), 'title': 'title-3', 'author': 'author-3', 'pages': 331, 'user': 'user-3', 'votes': ['user-8', 'user-2'], 'date': '2023-01-23', 'vote_count': 2}
{'_id': ObjectId('63c33fa19f340ce701cd82c9'), 'title': 'title-4', 'author': 'author-4', 'pages': 428, 'user': 'user-4', 'votes': ['user-7', 'user-0', 'user-0'], 'date': '2023-01-14', 'vote_count': 3}
{'_id': ObjectId('63c33fa19f340ce701cd82ca'), 'title': 'title-5', 'a

In [32]:
for x in bookclub.potential.collection.find({}):
    print(x)

{'_id': ObjectId('63c33fa19f340ce701cd82cf'), 'title': 'title-5', 'author': 'author-5', 'pages': 143, 'user': 'user-5', 'date': '2023-01-01', 'vote_count': 0, 'votes': []}
{'_id': ObjectId('63c33fa19f340ce701cd82d0'), 'title': 'title-4', 'author': 'author-4', 'pages': 428, 'user': 'user-4', 'date': '2023-01-14', 'vote_count': 0, 'votes': []}
{'_id': ObjectId('63c33fa19f340ce701cd82d1'), 'title': 'title-2', 'author': 'author-2', 'pages': 419, 'user': 'user-2', 'date': '2023-01-10', 'vote_count': 0, 'votes': []}


In [33]:
for x in bookclub.buffer.collection.find({}):
    print(x)

In [34]:
bookclub.book_of_month

{'_id': ObjectId('63c33fa19f340ce701cd82c6'),
 'title': 'title-1',
 'author': 'author-1',
 'pages': 234,
 'user': 'user-1',
 'votes': [],
 'date': '2023-01-25',
 'vote_count': 0}

In [26]:
bookclub.drop_all()